<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/LSTM_2_ipynbn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LSTM on Recipe DATA

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
import string
import re

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

#0 . Parameters

In [ ]:
VOCAB_SIZE = 10000
MAX_LEN= 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25


#1. Load the data

In [ ]:
#load the full dataset
with open("/app/data/epirecipes/full_format_recipes.json") as json_data:
  recipe_data = json.load(json_data)

In [ ]:
#Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [ ]:
#Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

In [ ]:
example = filtered_data[0]
print(example)

#2. Tokenise the data

In [ ]:
#Pad the punctuation to treat them as seperate 'words'

def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation}])", r" \1" , s)
  s = re.sub(" + " , " ", s)
  return s

text_data = [pad_punctuation(x) for x in filtered_data]

In [ ]:
#display an example of a recipe
example_data = text_data[9]
example_data


In [ ]:
#convert to a tensorflow dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [ ]:
#Create a vectorization layers
vectorize_layer = layers.TextVectorization(
    standardize = "lower",
    max_tokens = VOCAB_SIZE ,
    output_mode = "int",
    output_sequence_length = MAX_LEN + 1,
)

In [ ]:
#Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [ ]:
#Display some token:word mappings
for i , word in enumerate(vocab[:10]):
  print(f"{i} : {word}")

In [ ]:
#Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

#3. Create the training set

In [ ]:
#create the training set of the recipes and the same text shifted by one word
def prepare_inputs(text):
  text = tf.expand_dims(text, -1)
  tokenized_sentences = vectorize_layer(text)
  x = tokenized_sentences[:, : -1]
  y = tokenized_sentences[:, 1:]
  return x , y

train_ds = text_ds.map(prepare_inputs)

#4. Build the LSTM

In [ ]:
inputs = layers.Input(shape= (None,) , dtype = "int32")
x = layers.Embedding(VOCAB_SIZE , EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences = True)(x)
outputs = layers.Dense(VOCAB_SIZE , activation = "softmax")(x)
lstm = model.Model(inputs, outputs)
lstm.summary()

In [ ]:
import re

text = "Contact us at email@example.com or support@example.org for assistance."
pattern = r'\w+@\w+\.\w+'

matches = re.findall(pattern, text)
print(matches)

In [ ]:
if LOAD_MODEL:
  #model.load_weights('./models/model')
  lstm = models.load_model(".models/lstm" , compile = False)

#5. Train the LSTM

In [ ]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam" , loss_fn)

In [ ]:
#Create a TextGenrator checkpoint
class TextGenerator(callbacks.Callback):
  def __init__(self, inbex_to_word, top_k = 10):
    self.index_to_word = index_to_word
    self.word_to_index = {
        word: index for index, word in enumerate(index_to_word)
    }

  def sample_from(self, probs, temperature):
    probs = probs ** (1/ temperature)
    probs =  probs / np.sum(probs)
    return np.random.choice(len(probs), p = probs), probs

  def generate(self, start_prompt , max_tokens , temperature):
    start_tokens = [
        self.word_to_index.get(x,1) for x in start_prompt.split()
    ]
    sample_token = None
    info = []
    while len(start_tokens) < max_tokens and sample_token != 0:
      x = np.array([start_tokens])
      y = self.model.predict(x , verbose = 0)
      sample_token , probs = self.sample_from(y[0][-1] , temperature)
      info.append({"prompt" : start_prompt , "word_probs": probs})
      start_tokens.append(sample_token)
      start_prompt = start_prompt + "" + self.index_to_word[sample_token]
    print(f"\ngenrated text: \n{start_prompt}\n")
    return info

  def on_epoch_end(self, epoch , logs=None):
    self.generate("recipe for" , max_tokens = 100, temperature  = 1.0)

In [ ]:
#Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint/checkpoint.ckpt",
    save_weights_only = True,
    save_freq = "epoch",
    verbose = 0,
)

tensorboard_callback = callbacks.tensorBoard(log_dir = "./logs")

#tokenize starting prompt
text_genrator = TextGenerator(vocab)

In [ ]:
lstm.fit(
    train_ds,
    epochs =EPOCHS,
    callbacks = [model_checkpoint_callback, tensorboard_callback , text_genrator],
)

In [ ]:
#Save the final model
lstm.save("./models/lstm")

#6. Generate text using the LSTM

In [ ]:
def print_probs(info, vocab,  top_k = 5):
  for i in info:
    print(f"\nPROMPT: {i['prompt']}")
    word_probs = i["word_probs"]
    p_sorted = np.sort(word_probs)[::-1][:top_k]
    i_sorted = np.argsort(word_probs)[::-1][:top_k]
    for p, i in zip(p_sorted, i_sorted):
      print(f"{vocab[i]}: \t{np.round(100*p, 2)}%")
    print("-----------\n")

In [ ]:
info = text_genrator.generate(
    "recipe for roasted vegitables ! chop 1/", max_tokens = 10, temperature = 1.0
)

In [ ]:
print_probs(info , vocab)

In [ ]:
info = text_generator.generate(
    "recipe for roasted vegitables ! chop 1/", max_tokens = 10, temperature = 0.2
)

In [ ]:
print_probs(info , vocab)

In [ ]:
info = text_generator.generate(
    "recipe for chocolate ice cream | "  , max_tokens = 7, temperature = 1.0
)
print_probs(info , vocab)

In [ ]:
info = text_generator.generate(
    "recipe for chocolate ice Cream" , max_tokens = 7 , temperature = 0.2
)
print_probs(info, vocab)